In [1]:
import sqlalchemy as sa
import cx_Oracle
import xml.etree.ElementTree as ET
import json
import requests
from bs4 import BeautifulSoup


import pandas as pd
import time
import datetime
import random
random.uniform(0.2, 1.2)


0.9362678927039516

# data load

In [2]:
df = pd.read_csv("./data/kh_list_potential.csv")
df.head()

,key_title,key_rdate,key_href,key_cont
0,[K-Food] Ourhome tests potential of Korean foo...,"May 19, 2023",https://www.theinvestor.co.kr/view.php?ud=2023...,This article is the eighth installment in a s...
1,Samsung starts mass production of most advance...,"May 19, 2023",https://www.theinvestor.co.kr/view.php?ud=2023...,...ent of the 12nm-class process technology wa...
2,Korea bets on potential of advanced display te...,"May 18, 2023",https://www.theinvestor.co.kr/view.php?ud=2023...,The South Korean government is seeking to pus...
3,GS25 to debut ChatGPT-developed whiskey highball,"May 17, 2023",https://www.theinvestor.co.kr/view.php?ud=2023...,...uct was to cater to people in their 20s and...
4,"Thailand's PTT seeks investment, partnerships ...","May 17, 2023",https://www.theinvestor.co.kr/view.php?ud=2023...,Thailand's largest conglomerate as well as a ...


# 토큰화

In [3]:
from keras.preprocessing.text import Tokenizer
text_list = df['key_title']

token = Tokenizer()
token.fit_on_texts(text_list)
# print(' keras : \n')
# print('문장 카운트 : ',token.document_count) 

# print("==="*20)
# print("문장 토큰화(texts_to_sequences) : ")
# text_seq_list = token.texts_to_sequences(text_list)
# print(text_seq_list)

# print("==="*20)
# for seq, sent in zip (text_seq_list,text_list) :
#     print(sent, seq)

print("==="*20)      
print('단어 카운트 : \n', token.word_counts) 


# print("==="*20)
# print('각 단어에 매겨진 인덱스 값 :  \n',token.word_index)

# print("==="*20)
# print('각 단어가 몇 개의 문장에 포함되어 있는가 :  \n',token.word_docs) 


단어 카운트 : 
 OrderedDict([('k', 8), ('food', 6), ('ourhome', 1), ('tests', 1), ('potential', 11), ('of', 34), ('korean', 34), ('catering', 1), ('services', 2), ('on', 44), ('global', 14), ('stage', 1), ('samsung', 32), ('starts', 1), ('mass', 2), ('production', 6), ('most', 1), ('advanced', 4), ('dram', 2), ('chips', 11), ('korea', 49), ('bets', 5), ('display', 4), ('technologies', 2), ('gs25', 1), ('to', 163), ('debut', 1), ('chatgpt', 3), ('developed', 2), ('whiskey', 1), ('highball', 1), ("thailand's", 1), ('ptt', 1), ('seeks', 14), ('investment', 6), ('partnerships', 2), ('with', 28), ('tech', 10), ('firms', 13), ('ukraine', 5), ('asks', 1), ('businesses', 5), ('help', 2), ('rebuild', 1), ('war', 2), ('torn', 1), ('country', 1), ('electricity', 1), ('bills', 1), ('rise', 4), ('by', 11), ('5', 1), ('3', 1), ('as', 16), ('kepco', 1), ('battles', 1), ('mounting', 2), ('losses', 1), ('herald', 18), ('interview', 18), ('us', 23), ('states', 1), ('compete', 1), ('lure', 1), ('battery', 20)

In [4]:
word_counts=token.word_counts
type(word_counts)
# word_counts.keys()

collections.OrderedDict

# 표제어추출

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASIA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

<pre>
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\677\AppData\Roaming\nltk_data...

In [6]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASIA\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

<pre>
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\677\AppData\Roaming\nltk_data...

In [7]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

#------------ 'lives' -->  'life'  -------
# print('표제어 추출 전 :',word_counts.keys())
print("==="*20)
# print('표제어 추출 후 :',[lemmatizer.lemmatize(key) for key in word_counts.keys()])
lem_words=[lemmatizer.lemmatize(key) for key in word_counts.keys()]

##안 되는 거 같은데?? 

# 불용어

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASIA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords
stop_words_list = stopwords.words('english')
print('불용어 개수 :', len(stop_words_list))
print('불용어 10개 출력 :',stop_words_list[:10])

불용어 개수 : 179
불용어 10개 출력 : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [10]:
len(lem_words)

1612

In [11]:
new_words = []
for word in lem_words:
    if word not in(stop_words_list):
        new_words.extend(word)

In [12]:
len(lem_words)

1612

In [13]:
# 불용어 제거도 안돼 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ

# item 정렬

In [14]:
# word_counts.sort_values()
# sorted_dict = sorted(word_counts.items())
# sorted_dict
# item 오름차순 정렬

# 갯수 정렬 ()

In [15]:
sorted_dict = sorted(word_counts.items(), key = lambda item: item[1], reverse = True)
sorted_dict[:25]

[('to', 163),
 ('in', 89),
 ('for', 76),
 ('korea', 49),
 ('on', 44),
 ('of', 34),
 ('korean', 34),
 ('samsung', 32),
 ('market', 30),
 ('with', 28),
 ('sk', 24),
 ('us', 23),
 ('battery', 20),
 ('from', 19),
 ('at', 19),
 ('herald', 18),
 ('interview', 18),
 ('hyundai', 18),
 ('lg', 17),
 ('s', 17),
 ('new', 17),
 ('as', 16),
 ('chief', 16),
 ('2022', 15),
 ('global', 14)]

# 나오네!!!!!
<pre>
('samsung', 32)
('sk', 24)
('battery', 20)
('herald', 18)
('hyundai', 18),
('lg', 17)
